In [1]:
!pip install transformers -U
!pip install peft
!pip install -U langchain-chroma
!pip install sentence-transformers

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.47.1 requires tokenizers<0.22,>=0.21, but you have tokenizers 0.20.3 which is incompatible.



  Using cached tokenizers-0.20.3-cp310-none-win_amd64.whl.metadata (6.9 kB)
Using cached tokenizers-0.20.3-cp310-none-win_amd64.whl (2.4 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.0
    Uninstalling tokenizers-0.21.0:
      Successfully uninstalled tokenizers-0.21.0
  Using cached tokenizers-0.21.0-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
Using cached tokenizers-0.21.0-cp39-abi3-win_amd64.whl (2.4 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chromadb 0.5.23 requires tokenizers<=0.20.3,>=0.13.2, but you have tokenizers 0.21.0 which is incompatible.


In [2]:
from langchain_chroma import Chroma
import torch
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain_chroma import Chroma # Importing Chroma vector store from Langchain
from langchain_core.prompts import PromptTemplate
from peft import PeftModel, PeftConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
import numpy as np
import re


c:\Users\bhks0\anaconda3\envs\eunbin\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1. 저장된 학습 모델 로딩하기
  - Foundation 모델, PEFT Adapter

In [3]:
# 기본 모델과 PEFT 어댑터 경로 설정
base_model_path = r"C:\Users\bhks0\Desktop\LLM 응용1\models\Foundation_model"    # 기본 LLM 모델이 저장된 경로
peft_adapter_path = r"C:\Users\bhks0\Desktop\LLM 응용1\models\Tuning_model"  # 로컬에 저장된 PEFT 어댑터 경로

# 기본 모델 로드
tokenizer = AutoTokenizer.from_pretrained(base_model_path)
base_model = AutoModelForCausalLM.from_pretrained(base_model_path)

base_model.resize_token_embeddings(len(tokenizer))  # tokenizer에서 사용하는 어휘 크기

# PEFT 설정 로드
peft_config = PeftConfig.from_pretrained(peft_adapter_path, local_files_only=True)

# 기본 모델에 PEFT 어댑터 적용
model = PeftModel.from_pretrained(base_model, peft_adapter_path, local_files_only=True)

# GPU 사용 설정 (GPU가 있을 경우)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPT2LMHeadModel(
      (transformer): GPT2Model(
        (wte): Embedding(51201, 768)
        (wpe): Embedding(1024, 768)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-11): 12 x GPT2Block(
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2SdpaAttention(
              (c_attn): Conv1D(nf=2304, nx=768)
              (c_proj): lora.Linear(
                (base_layer): Conv1D(nf=768, nx=768)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=768, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=768, bias=False)
                )
                (lora_embedding_A): ParameterDict()
           

In [4]:
from transformers import AutoConfig

base_model_config = AutoConfig.from_pretrained(base_model_path)
peft_adapter_config = PeftConfig.from_pretrained(peft_adapter_path, local_files_only=True)

print(f"Base model architecture: {base_model_config.model_type}")
print(f"PEFT adapter target modules: {peft_adapter_config.target_modules}")


Base model architecture: gpt2
PEFT adapter target modules: {'c_proj'}


2. Query 증강
  - 벡터 DB의 저장된 내용과 Query의 유사도로 관련 문장 추출

In [5]:
from langchain.embeddings import HuggingFaceEmbeddings

CHROMA_PATH = "./chroma"

modelPath = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2" # open source embedding model 

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device': 'cpu'}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': True}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)

def clean_response(text):
    """Remove duplicate words, sentences, and excessive repetition."""
    sentences = text.split(". ")  # 문장 단위로 분리
    unique_sentences = list(dict.fromkeys(sentences))  # 중복 제거
    cleaned_text = ". ".join(unique_sentences).strip()

    # 특정 단어나 문장의 반복 제거
    excessive_patterns = re.findall(r"(\b\w+\b)( \1)+", cleaned_text)  # 반복 단어 감지
    for pattern in excessive_patterns:
        cleaned_text = re.sub(rf"{pattern[0]}( {pattern[0]})+", pattern[0], cleaned_text)

    return cleaned_text


def query_rag(query_text):
    embedding_function = embeddings

    # Load the existing Chroma database
    db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

    # Retrieve context from the DB using similarity search
    results = db.similarity_search_with_relevance_scores(query_text, k=2)

    # Print the results for debugging
    print(f"Search Results: {results}")  # 확인용 출력
    
    # 유사도 점수 필터링 (-2에서 +2까지 허용)
    valid_results = [res for res in results if -2 <= res[1] <= 2]
    if not valid_results:
        print("No valid search results.")
        return "관련된 응답을 찾을 수 없습니다."

    # 가장 높은 유사도 점수를 가진 결과 선택
    best_result = max(valid_results, key=lambda x: x[1])
    return best_result[0].page_content

C:\Users\bhks0\AppData\Local\Temp\ipykernel_24460\523530696.py:14: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


3. 증강된 검색어로 모델 Inference

In [6]:
def model_run(context_text, query_text, prompt_template):
    # Combine context from matching documents
    # context_text = " ".join([doc.page_content for doc, _score in context_text])
    context_text = " ".join([context_text])

    # Create prompt template using context and query text
    prompt_template = PromptTemplate.from_template(prompt_template)
    prompt = prompt_template.format(context=context_text, question=query_text)
    print(f"Prompt: {prompt}")  # 디버깅

    # pad_token_id를 eos_token_id로 설정
    if tokenizer.pad_token is None:
      tokenizer.pad_token = tokenizer.eos_token
    
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(device)
    
    inputs.pop("token_type_ids", None)
    
    # 모델 추론 (생성 결과)
    with torch.no_grad():
      outputs = model.generate(
        input_ids=inputs["input_ids"], 
        attention_mask=inputs['attention_mask'], 
        pad_token_id=tokenizer.pad_token_id,
        max_length=256, num_return_sequences=1,
        no_repeat_ngram_size=3,  # 3그램 반복 방지
        repetition_penalty=2.0,   # 반복 생성 억제
        temperature=0.7  #샘플링의 다양성 설정
        )

    # 결과 디코딩 및 출력
    response_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return clean_response(response_text)


In [7]:
query_text = "눈물이 자꾸 나요."

# Prompt template
prompt_template = """
Context: {context}
Question: {question}
"""

# 1. RAG로 컨텍스트 검색 및 번역
context_text = query_rag(query_text)

if context_text:
    # 2. 모델 추론
    response_text = model_run(context_text, query_text, prompt_template)
    
    # 3. 출력 정리
    print("=== 모델의 응답 ===")
    
    # # 모델의 응답 부분만 추출하고 "Answer:" 추가
    response_lines = response_text.strip().split("\n")
    
    # # 가장 마지막 줄이 모델의 답변이라고 가정하고 추출
    answer = " ".join(response_lines[2:]).strip()  # 리스트를 공백으로 연결
    
    # 'Answer:'를 붙여서 출력
    print(f"Context: {context_text}\n")  # context_text를 출력
    print(f"Question: {query_text}\n")  # query_text를 출력
    print(f"Answer: {answer}\n")  # 모델의 답변을 출력 
    
else:
    print("No relevant context found in the vector database.")


C:\Users\bhks0\AppData\Local\Temp\ipykernel_24460\523530696.py:41: UserWarning: Relevance scores must be between 0 and 1, got [(Document(metadata={'page': 130, 'source': 'C:\\Users\\bhks0\\Desktop\\LLM 응용1\\Document\\멀티턴심리상담.pdf', 'start_index': 1202}, page_content='해보았지만 해결책이 되지 않아 더욱 괴로워하고 계신 것 같습니다 이 기분을 다스리는 것이 어렵겠지만 지금 당장 내다보는 것이 중요합니다 어떻게 하면 지금 이 기분을 풀어나갈 수 있을지 먼저 그것을 생각해보세요'), -1.5608273426564572), (Document(metadata={'page': 9, 'source': 'C:\\Users\\bhks0\\Desktop\\LLM 응용1\\Document\\멀티턴심리상담.pdf', 'start_index': 1005}, page_content='도움이 될까요 그렇다면 깊이 있게 이야기해보겠습니다 지금 내담자님은 고민 중이고 불안하신 마음을 갖고 계시군요 저는 내담자님의 상황을 충분히 이해하고 있으며 불안한 마음이 지속되면 정'), -1.6705286772471588)]
  results = db.similarity_search_with_relevance_scores(query_text, k=2)
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
c:\Users\bhks0\anaconda3\envs\eunbin\lib\site-packages\transformers\generation\configuration_utils.py:628: UserW

Search Results: [(Document(metadata={'page': 130, 'source': 'C:\\Users\\bhks0\\Desktop\\LLM 응용1\\Document\\멀티턴심리상담.pdf', 'start_index': 1202}, page_content='해보았지만 해결책이 되지 않아 더욱 괴로워하고 계신 것 같습니다 이 기분을 다스리는 것이 어렵겠지만 지금 당장 내다보는 것이 중요합니다 어떻게 하면 지금 이 기분을 풀어나갈 수 있을지 먼저 그것을 생각해보세요'), -1.5608273426564572), (Document(metadata={'page': 9, 'source': 'C:\\Users\\bhks0\\Desktop\\LLM 응용1\\Document\\멀티턴심리상담.pdf', 'start_index': 1005}, page_content='도움이 될까요 그렇다면 깊이 있게 이야기해보겠습니다 지금 내담자님은 고민 중이고 불안하신 마음을 갖고 계시군요 저는 내담자님의 상황을 충분히 이해하고 있으며 불안한 마음이 지속되면 정'), -1.6705286772471588)]
Prompt: 
Context: 해보았지만 해결책이 되지 않아 더욱 괴로워하고 계신 것 같습니다 이 기분을 다스리는 것이 어렵겠지만 지금 당장 내다보는 것이 중요합니다 어떻게 하면 지금 이 기분을 풀어나갈 수 있을지 먼저 그것을 생각해보세요
Question: 눈물이 자꾸 나요.

=== 모델의 응답 ===
Context: 해보았지만 해결책이 되지 않아 더욱 괴로워하고 계신 것 같습니다 이 기분을 다스리는 것이 어렵겠지만 지금 당장 내다보는 것이 중요합니다 어떻게 하면 지금 이 기분을 풀어나갈 수 있을지 먼저 그것을 생각해보세요

Question: 눈물이 자꾸 나요.

Answer: 네. 너무 힘들어요. 그래서 엄마가 울고 있어요? 아니면 엄마랑 같이 있으면 좀 나아져야 할 텐데. 그래도 안 풀려서 더 힘들었어요? - 그냥 괜찮아요, 그냥 참아야죠, 하고 싶어서 그래봤자